In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def request_load(_id):
    session = requests.Session()
    timeout = 10
    headers = {
        'authority': 'txdot.public.ms2soft.com',
        'accept': '*/*',
        'accept-language': 'en,zh;q=0.9,zh-CN;q=0.8,ja;q=0.7,zh-TW;q=0.6',
        'referer': 'https://txdot.public.ms2soft.com/tcds/tdetail.asp?updatemap=&from_map=',
        'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    }
    res = session.get('https://txdot.public.ms2soft.com/tcds/tsearch.asp', headers = headers) #request for home page and store the cookies in cks
    cks = session.cookies

    lst = [] #create an empty list to append all responses
    
    for pg in range(1,6):  #AADT detail has at most 5 pages
        params = {
            'offset': '0',
            'agency_id': '97',
            'local_id': _id,
            'page_type': '',
            'pg': str(pg),
        }

        response = requests.get(
            'https://txdot.public.ms2soft.com/tcds/ajax/tcds_tdetail_aadt.asp',
            params=params,
            timeout = timeout,
            headers=headers,
            cookies = cks
        )
        
        lst.append(response.text)
        
    return lst
    

    

In [ ]:
station = "43H3"
lst = request_load(station)

In [ ]:
col_nm = []
row_data = []

for response in lst:
    soup = BeautifulSoup(response, 'html.parser')
    rows_tr = soup.find_all('tr', class_ = 'FormRowLabel')

    # Getting column names
    # if col_nm:
    #     pass
    # else:
    #     for cell in rows_tr[0]:
    #         if cell.text.strip() != '':
    #             col_nm.append(cell.text.strip())
    if not col_nm:
        col_nm = [cell.text.strip() for cell in rows_tr[0] if cell.text.strip()]
    
    # Getting values by rows
    for i in range(1, len(rows_tr)):
        row = rows_tr[i].find_all('td')
        if len(row) <= 1:
            break
        cell_data = [''.join(str(item) for item in td.contents if not getattr(item, 'name', None) == 'sup').strip() for td in row if td.contents]
        row_data.append(cell_data)
        

    # for i in range(len(rows_tr)-1):
    #     row = rows_tr[i+1].find_all('td')
    #     if len(row) <= 1:
    #         break
    #     data = []
    #     for cell_bs in row:
    #         cell = cell_bs.text
    #         if cell != '':
    #             data.append(cell)
    #     row_data.append(data)   


In [ ]:
row_data

In [ ]:
aadt